***FCIM.FIA - Fundamentals of Artificial Intelligence***

> **Lab 1:** *Expert Systems* \\
> **Performed by:** *Bajenov Sevastian*, group *FAF-213* \\
> **Verified by:** Elena Graur, asist. univ.

## Imports and Utils

Create a virtual environment, install all the necessary dependencies so that you can run the notebook using your virtual environment as a kernel.

In [ ]:
# pip install -r requirements.txt

## Task 1

In order to develop the expert system, I started from defining the knowledge base. The system includes 5 main hypotheses, or tourist types and one additional (Luna City citizen). The main ones can be divided into 2 groups: `poor` and `rich` tourists. The first category includes `Sugar Daddy`, `Influencer` and `Drug Dealer`. The second one consists of the `Crypto Investor` and `Student`. The Luna Citizen, or `Lunatic` is an additional hypothesis, having the most simple proof path.

Below is presented the goal tree describing all the facts and the associated hypotheses:

![GoalTree](./goal_tree.png)

## Task 2

The `LUNA_GUESTS_RULES` defines the connection between the intermediary facts and hypotheses and the initial facts. Each rule includes only two conditions and there is only one rule which is of type `OR`. The rest of the rules are of the `AND` type. It is important to mention one convention regarding the structure of each rule. All of the intermediary facts are placed as the first condition in each of the rules. It helps further with the `interactive traversal` of the goal tree.

In [21]:
from production import IF, AND, THEN, OR

LUNA_GUESTS_RULES = (

    IF( OR( '(?x) floats in the air',
            '(?x) has shapeless body' ),
        THEN( '(?x) is a Lunatic' )),
    
    IF( AND( '(?x) wears jewellery',
             '(?x) has expensive accesories' ),
        THEN( '(?x) is a rich person supposedly' )),

    IF( AND( '(?x) wears baggy clothes',
             '(?x) has small luggage' ),
        THEN( '(?x) is a poor person' )),
    
    IF( AND( '(?x) is a rich person supposedly', 
            '(?x) has personal interspace vehicle'),
        THEN( '(?x) is a rich person' )),
   
    IF( AND( '(?x) is a rich person',
             '(?x) is accompanied by a very young lady' ),
        THEN( '(?x) is a Sugar Daddy' )),
   
    IF( AND( '(?x) is a rich person',
             '(?x) is live on social media' ),
        THEN( '(?x) is an Influencer' )),
    
    IF( AND( '(?x) is a rich person',
             '(?x) smokes expensive cigars' ),
        THEN( '(?x) is a Drug Dealer' )),
    
    IF( AND( '(?x) is a poor person',
             '(?x) has an addicted behavior' ),
        THEN( '(?x) is a Crypto Investor' )),
    
    IF( AND( '(?x) is a poor person', 
             '(?x) is in the group of similar looking people' ),
        THEN( '(?x) is a Student' )),
    )

## Task 3

The first algorithm which is needed for implementing the expert system is the `forward chaining` algorithm. It is also known as a forward deduction or forward reasoning method when using an inference engine. Forward chaining is a form of reasoning which start with initial facts in the knowledge base and applies inference rules in the forward direction to extract more data until a goal is reached.

The algorithm provided in the conditions of the laboratory work is rather straightforward. It iterates over the list of rules and tries to apply them to each of the facts in the provided list until the algorithm reaches an intermediary fact or hypothesis. Below is presented verbose output of the `forward_chain` method and it can be observed that the algorithm aggregates facts consequently until the hypothesis is reached. There is also printed the difference between the initial data and the data obtained after performing forward chaining over the goal tree.

In [28]:
from rules import LUNA_GUESTS_RULES

def forward_chain(rules, data, apply_only_one=False, verbose=False):
    old_data = ()

    while set(old_data) != set(data):
        old_data = list(data)
        for condition in rules:
            data = condition.apply(data, apply_only_one, verbose)
            if set(data) != set(old_data):
                break

    return set(data)

facts = ['Tim wears baggy clothes', 'Tim has small luggage', 'Tim has an addicted behavior']
resulting_facts = forward_chain(LUNA_GUESTS_RULES, facts, verbose=True)
print(f"\n{resulting_facts - set(facts)}")

Rule: IF(AND('(?x) wears baggy clothes', '(?x) has small luggage'), THEN('(?x) is a poor person'))
Added: Tim is a poor person
Rule: IF(AND('(?x) is a poor person', '(?x) has an addicted behavior'), THEN('(?x) is a Crypto Investor'))
Added: Tim is a Crypto Investor

{'Tim is a poor person', 'Tim is a Crypto Investor'}


## Task 4

The second essential algorithm for the expert system development is the `backward chaining` algorithm. It is a form of reasoning, which starts with the goal and works backward, chaining through rules to find known facts that support the goal. 

In my implementation, the algorithm retrieves the `antecedent` (conditions) and `consequent` (conclusion) of each rule. If the `consequent` matches the `hypothesis`, the program tries to prove the `antecedent`. If the antecedent involves an `AND`, we try to prove recursively all the subgoals. If it involves an `OR`, then we prove at least one subgoal. Otherwise, the antecedent is treated as a single condition. Finally, the `backward_chain` method is being called recursively to prove each subgoal of the antecedent.

Moreover, the print statements are arranged in such way that each of the subgoals, as well as the hypothesis itself, are printed on new lines without indent, and their antecedents are presented below with an indent (also the type of the condition is printed).

In [23]:
import re

from rules import LUNA_GUESTS_RULES
from production import match, AND, OR

def backward_chain(rules, hypothesis, examined_person, verbose=False, depth=0):
    if hypothesis in rules:
        if verbose:
            print(f"Found {hypothesis} in facts.")
        return [hypothesis]

    goal_tree = []
    for rule in rules:
        antecedent = rule.antecedent()
        consequent = rule.consequent()

        # If the rule's consequent matches the hypothesis, try to prove the antecedent.
        if match(consequent[0], hypothesis):
            if verbose:
                hypothesis = re.sub(r'\(\?x\)', examined_person, hypothesis)
                condits = [re.sub(r'\(\?x\)', examined_person, condition) for condition in antecedent.conditions()]

                print(f"{' ' * depth}? {hypothesis}")
                depth += 2

                if 'AND' in str(type(antecedent)):
                    print(f"{' ' * depth}- {condits[0]} and {condits[1]}")
                else:
                    print(f"{' ' * depth}- {condits[0]} or\n{' ' * depth}- {condits[1]}")
                
                depth += 2

            # Check if the antecedent can be proven recursively
            if isinstance(antecedent, AND):
                subgoals = [backward_chain(rules, subgoal, examined_person, verbose) for subgoal in antecedent]
                goal_tree.append(AND(*subgoals))
            elif isinstance(antecedent, OR):
                subgoals = [backward_chain(rules, subgoal, examined_person, verbose) for subgoal in antecedent]
                goal_tree.append(OR(*subgoals))
            else:
                # Single condition antecedent
                subgoal = backward_chain(rules, antecedent, verbose)
                goal_tree.append(subgoal)

    return goal_tree or [hypothesis]

hypothesis = 'Tim is a Student'
goal_tree = backward_chain(LUNA_GUESTS_RULES, hypothesis, hypothesis.split()[0], verbose=True)
print(f"\n{goal_tree}")

? Tim is a Student
  - Tim is a poor person and Tim is in the group of similar looking people
? Tim is a poor person
  - Tim wears baggy clothes and Tim has small luggage

[AND([AND(['(?x) wears baggy clothes'], ['(?x) has small luggage'])], ['(?x) is in the group of similar looking people'])]


## Task 5

An important step in developing an interactive expert system consists in creating the algorithm for generating user questions. I decided to ask the user 3 types of questions: `yes/no`, `multiple choice` and `user input` which is basically the prompt which requires the user to enter a fact about the person being analyzed. Therefore, I created a class `QuestionGenerator` with 3 methods implementing the question generation based on the rule conditions:

1. **generate_yes_no_question(self, condition)**

    According to the english grammar rules converts the sentence into a general question which requires yes or no answer. The `convert_verb_to_infinitive` method helps to change the verb's tense in the original sentence.

2. **generate_multiple_choice_question(self, conditions)**

    Iterates over the list of conditions and adds them as options into a numerated list. The last option in the resulting list is `None of the listed options`. The user is then prompted to choose one of the options.

3. **generate_user_input_question(self)**
    
    Provides a prompt which simply asks the user to give an insight about the person, he/she tries to identify.

In [24]:
class QuestionGenerator:

    question_placeholder = 'the person'

    def convert_verb_to_infinitive(self, verb):
        if verb == 'has':
            return 'have'
        else:
            return verb[:-1]


    def generate_yes_no_question(self, condition):
        words = condition.split()
        verb = words[1]

        if verb.startswith('is'):
            question = f"Is {self.question_placeholder} {' '.join(words[2:])}?"
        else:
            question = f"Does {self.question_placeholder} {self.convert_verb_to_infinitive(verb)} " + \
                f"{' '.join(words[2:])}?"
        
        return question
    

    def generate_multiple_choice_question(self, conditions):
        question = "The person you are trying to identify...\n\n"
        
        index = 1
        for condition in conditions:
            question += f"{index}. ...{' '.join(condition.split()[1:])}\n"
            index += 1

        question += f"{index}. None of the given options"

        return question


    def generate_user_input_question(self):
        question = f"Provide an additional insight about {self.question_placeholder}.\n"
        question += "For example, 'is live on social media'."

        return question
    

question_generator = QuestionGenerator()
print(question_generator.generate_yes_no_question('(?x) is accompanied by a very young lady'))
print('------------------------------------------------')
print(question_generator.generate_multiple_choice_question(['(?x) is accompanied by a very young lady', 'is live on social media']))
print('------------------------------------------------')
print(question_generator.generate_user_input_question())

Is the person accompanied by a very young lady?
------------------------------------------------
The person you are trying to identify...

1. ...is accompanied by a very young lady
2. ...live on social media
3. None of the given options
------------------------------------------------
Provide an additional insight about the person.
For example, 'is live on social media'.


## Task 6-7

Before analyzing the interactive expert system implementation, it is necessary to go through the helper methods of the `ExpertSystem` class:

1. **System setup methods**

    The two methods from this category identify hypotheses and leaf/intermediary rules at the start of the interactive goal tree traversal. `Leaf rules` are those rules, having conditions which are not contained by any rule's action. The rest of the rules are `intermediary` ones.

In [ ]:
# do not run this cell

def identify_hypotheses(self, actions):
    for action in actions:
        invalid_action = False

        for rule in LUNA_GUESTS_RULES:
            if action in rule._conditional.conditions():
                invalid_action = True
                break
        
        if not invalid_action:
            self.hypotheses.append(action)


def identify_leaf_and_intermediary_rules(self, actions):
    for rule in LUNA_GUESTS_RULES:
        invalid_condition = False

        for condition in rule._conditional.conditions():
            if condition in actions:
                invalid_condition = True
                self.intermediary_rules.append({'rule': rule, 'checked': False, 'leaf': False})
                break
        
        if not invalid_condition:
            self.leaf_rules.append({'rule': rule, 'checked': False, 'leaf': True})

2. **Question asking methods**

    These methods call the corresponding methods from the `QuestionGenerator` class and collect the user response which is either an integer or a string.

In [ ]:
# do not run this cell

def ask_single_choice_question(self, condition):
    question = self.question_generator.generate_yes_no_question(condition)
    print(f"\n{question}")
    user_response = input("\nYes/No? ")

    while (not user_response.startswith("Yes")) and (not user_response.startswith("No")):
        user_response = input("\nYes/No? ")
    
    return user_response


def ask_multiple_choice_question(self, conditions):
    question = self.question_generator.generate_multiple_choice_question(conditions)
    print(f"\n{question}")
    user_response = int(input("\nChoose an option: "))

    while not user_response in [option for option in range(1, len(conditions) + 2)]:
        user_response = int(input("\nChoose an option: "))
    
    return user_response


def ask_user_input_question(self):
    question = self.question_generator.generate_user_input_question()
    print(f"\n{question}")
    user_response = input("\nYour insight: the person ")

    return user_response

3. **Rule status tracking methods**

    First of all, this set of methods helps us track the rules which were already verified by the expert system. Secondly, depending on the situation the methods extract either a single rule or several rules at once in case if the system formulates multiple choice question and needs several rules for a single fact to process.

In [ ]:
# do not run this cell

def get_a_new_unchecked_rule(self, rules, existing_fact=None):
    for rule in rules:
        if not rule['checked']:
            if (existing_fact is None) or (existing_fact in rule['rule']._conditional.conditions()):
                return rule
    return None


def get_list_of_unchecked_rules(self, rules, existing_fact):
    aggregated_rules = []

    for rule in rules:
        if not rule['checked']:
            if existing_fact in rule['rule']._conditional.conditions():
                aggregated_rules.append(rule)

    return aggregated_rules


def mark_rule_as_checked(self, checked_rule):
    rules = []
    if checked_rule['leaf']:
        rules = self.leaf_rules  
    else:
        rules = self.intermediary_rules

    for rule in rules:
        if rule['rule'].__str__().startswith(checked_rule['rule'].__str__()):
            rule['checked'] = True
            return

Further on, it is important to explain how the rules are being verified (note: both methods explained here return the list of newly obtained facts). In case of the `multiple choice question` the program asks the user either the corresponding question or the user input question. The probability for the latter to occur is lower. The fact entered by the user may be then added to the `facts` list which already contains the `existing fact` to which the conditions being verified are linked. 

The situation with the single condition verification is a little bit more complex. First of all, the program distinguishes `OR` and `AND` rule types to avoid unnecessary condition check. Also, if the rule which is being checked is an intermediary one then its intermediary condition (fact) is not being checked. The resulting logic is encapsulated into a chain of if/else statements. Necessary note: the program assumes that each rule contains only 2 conditions.

In [ ]:
# do not run this cell

def verify_single_condition_fulfillment(self, rule, existing_fact):
    facts = set()
    conditions = rule['rule']._conditional.conditions()

    if existing_fact is not None:
        facts.add(existing_fact)

        if 'OR' in str(type(rule['rule']._conditional)):
            return facts
        
        response = self.ask_single_choice_question(conditions[1])
        if response.startswith("Yes"):
            facts.add(conditions[1])
    else:
        first_response = self.ask_single_choice_question(conditions[0])
        if first_response.startswith("Yes"):
            facts.add(conditions[0])
            
            if 'OR' in str(type(rule['rule']._conditional)):
                return facts
        else:
            if 'AND' in str(type(rule['rule']._conditional)):
                return facts
            
        second_response = self.ask_single_choice_question(conditions[1])
        if second_response.startswith("Yes"):
            facts.add(conditions[1])
    
    return facts


def verify_multiple_conditions_fulfillment(self, rules, existing_fact):
    facts = set()
    facts.add(existing_fact)

    conditions = []
    for rule in rules:
        conditions.append(rule['rule']._conditional.conditions()[1])

    question_choice = randint(1, 3)

    if question_choice != 1:
        response = self.ask_multiple_choice_question(conditions)

        if response != len(conditions) + 1:
            facts.add(conditions[response - 1])
    else:
        response = self.ask_user_input_question()

        for condition in conditions:
            if condition.endswith(response):
                facts.add(condition)
                break

    return facts

Finally, it is time to explain the general flow of the interactive goal tree traversal. The algorithm works roughly like follows:

1. Identify hypotheses and leaf/intermediary rules;
2. While no return or break instructions occur proceed:
    
    2.1 Get a new unchecked rule from the list of leaf rules;

    2.2 If `current_rule` is None than the system failed to identify the person (exit);
    
    2.3 While `current_rule` is not None or there are no return or break instructions proceed:
        
    2.3.1 Verify single condition fulfillment or set the `facts` equal to the facts obtained from the multiple choice question;

    2.3.2 Obtain `new_facts` using `forward_chain` method and mark rule as checked;

    2.3.3 Try to obtain an intermediary fact by finding the difference `new_facts - facts`;

    2.3.4 If the difference is an empty set: if the rule is a leaf one then exit the while loop, else get a new unchecked rule using current intermediary fact; else check if the resulting fact is in `hypotheses`, then exit completely, else verify multiple conditions of the current intermediary fact;

In [ ]:
# do not run this cell

def traverse_goal_tree_interactively(self):
    if len(self.actions) == 0:
        self.actions = [rule._action[0] for rule in LUNA_GUESTS_RULES]

    if len(self.hypotheses) == 0:
        self.identify_hypotheses(self.actions)

    self.identify_leaf_and_intermediary_rules(self.actions)

    while True: 
        existing_fact = None
        current_rule = self.get_a_new_unchecked_rule(self.leaf_rules)

        if current_rule is None:
            print("\nThe person cannot be identified by the system")
            return
        
        choice_facts = None

        while current_rule is not None:
            if choice_facts is not None:
                facts = choice_facts
            else:
                facts = self.verify_single_condition_fulfillment(current_rule, existing_fact)
            
            new_facts = forward_chain(LUNA_GUESTS_RULES, facts)
            self.mark_rule_as_checked(current_rule)

            try:
                intermediary_fact = (new_facts - facts).pop()

                if intermediary_fact in self.hypotheses:
                    print(f"\nThe person {' '.join(intermediary_fact.split()[1:])}")
                    return
                else:
                    existing_fact = intermediary_fact
                    curr_rules = self.get_list_of_unchecked_rules(self.intermediary_rules, existing_fact)

                    if len(curr_rules) > 1:
                        choice_facts = self.verify_multiple_conditions_fulfillment(curr_rules, existing_fact)
                        for rule in curr_rules:
                            self.mark_rule_as_checked(rule)
                    else:
                        current_rule = curr_rules[0]

            except (KeyError):
                if current_rule['leaf']:
                    break
                else:
                    current_rule = self.get_a_new_unchecked_rule(self.intermediary_rules, list(facts)[0])

Last remark: the method for printing encyclopedia view of a hypothesis using backward chaining simply prompts the user to enter a hypothesis and then calls `backward_chain` method with verbose output.

In [ ]:
# do not run this cell

def get_person_category_encyclopedia_view(self):
    if len(self.actions) == 0:
        self.actions = [rule._action[0] for rule in LUNA_GUESTS_RULES]

    if len(self.hypotheses) == 0:
        self.identify_hypotheses(self.actions)

    hypothesis = input("\nEnter your hypothesis (for example, 'Tim is a Student'): ")
    words = hypothesis.split()
    words[0] = '(?x)'
    hypothesis_action = ' '.join(words)

    while hypothesis_action not in self.hypotheses:
        print("\nThe system does not recognize the hypothesis")
        hypothesis = input("\nTry again (for example, 'Tim is a Student'): ")
        words = hypothesis.split()
        words[0] = '(?x)'
        hypothesis_action = ' '.join(words)

    print()
    backward_chain(LUNA_GUESTS_RULES, hypothesis, hypothesis.split()[0], verbose=True)

Below is presented an example of using the system in two modes (based on forward and backward chaining respectively):

In [26]:
from expert_system import ExpertSystem

if __name__=='__main__':
    print("Welcome to Luna City Tourist Expert System!")

    while True:
        request = int(input("\nChoose expert system mode:\n1 - person identification\n2 - encyclopedia view\n3 - exit\n\n"))

        while request not in [1, 2, 3]:
            request = int(input("\nChoose expert system mode:\n1 - person identification\n2 - encyclopedia view\n\n"))
        
        es = ExpertSystem()
        if request == 1:
            es.traverse_goal_tree_interactively()
        elif request == 2:
            es.get_person_category_encyclopedia_view()
        else:
            print("\nThank you for using our expert system")
            break

Welcome to Luna City Tourist Expert System!

Does the person float in the air?

Does the person have shapeless body?

Does the person wear jewellery?

Does the person have expensive accesories?

Does the person have personal interspace vehicle?

The person you are trying to identify...

1. ...is accompanied by a very young lady
2. ...is live on social media
3. ...smokes expensive cigars
4. None of the given options

The person is an Influencer

? Tim is a Crypto Investor
  - Tim is a poor person and Tim has an addicted behavior
? Tim is a poor person
  - Tim wears baggy clothes and Tim has small luggage

Thank you for using our expert system


## Conclusions:

In this laboratory work I learned the concept of expert systems in artificial intelligence. I studied thouroughly forward and backward chaining algorithms and applied them to develop an interactive expert system. During the process of its development I used iterative approach, but it is also possible to use recursive approach. In my opinion, recursion could be a better option which could help avoid redundant checks. Moreover, in my implementation I did not use backward chaining to ask questions interactively, however, it could be another option of optimization.

## Acknowledgements

In this laboratory work I was assisted by Arteom Kalamaghin from FAF-211. He helped me solve ambiguity which occured when I designed the goal tree and also explained to me how multiple choice questions should be correctly formulated.

## Bibliography:

1. https://www.javatpoint.com/forward-chaining-and-backward-chaining-in-ai
2. https://towardsdatascience.com/are-expert-systems-dead-87c8d6c26474